<a href="https://colab.research.google.com/github/shubham8garg/MCP_A2A_session/blob/main/MCP_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 MCP (Model Context Protocol) — Live Demo

This notebook has **two sections**:

| Section | What | Who |
|---------|------|-----|
| **🎯 SECTION A: See the Magic** | Run & compare results — no code reading needed | Everyone |
| **🔬 SECTION B: Under the Hood** | See how each piece works in detail | Curious minds |

**Instructions:** Just run each cell top to bottom. Section A takes ~3 minutes.

In [ ]:
!pip install openai mcp fastmcp httpx python-dotenv -q

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-"
# Verify
assert os.environ.get("OPENAI_API_KEY", "").startswith("sk-"), "❌ Please set a valid OPENAI_API_KEY"
print("✅ OPENAI_API_KEY is set")

✅ OPENAI_API_KEY is set


In [ ]:
# ── Setup: imports, client, helpers, and MCP server file ──
import os, json, time, asyncio, sys, subprocess
from openai import OpenAI
from mcp.client.session import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from mcp.client.stdio import stdio_client, StdioServerParameters

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
MODEL = "gpt-5.2"

def print_header(text):
    width = 60
    print("\n" + "═" * width)
    print(f"  {text}")
    print("═" * width)

def print_tool_call(name, args):
    print(f"  📤 TOOL CALL: {name}")
    if isinstance(args, str): args = json.loads(args)
    for k, v in args.items():
        print(f"     {k}: {v}")

def mcp_tools_to_openai_functions(mcp_tools):
    openai_tools = []
    for tool in mcp_tools:
        openai_tools.append({
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description or "",
                "parameters": tool.inputSchema if tool.inputSchema else {"type": "object", "properties": {}},
            },
        })
    return openai_tools

async def run_agentic_loop(session, messages, openai_tools, max_iterations=10):
    stats = {"iterations": 0, "tool_calls": 0, "tools_used": set()}
    for iteration in range(1, max_iterations + 1):
        stats["iterations"] = iteration
        print(f"\n🔄 Iteration {iteration}")
        print("─" * 45)
        iter_start = time.time()
        response = client.chat.completions.create(model=MODEL, messages=messages, tools=openai_tools if openai_tools else None)
        choice = response.choices[0]
        message = choice.message
        print(f"  ⏱️  LLM responded in {time.time() - iter_start:.1f}s | Finish: {choice.finish_reason}")
        if choice.finish_reason == "tool_calls" and message.tool_calls:
            num_calls = len(message.tool_calls)
            stats["tool_calls"] += num_calls
            print(f"  🔧 LLM wants {num_calls} tool call(s)")
            messages.append(message.model_dump())
            for tool_call in message.tool_calls:
                func_name = tool_call.function.name
                func_args = json.loads(tool_call.function.arguments)
                stats["tools_used"].add(func_name)
                print()
                print_tool_call(func_name, func_args)
                try:
                    mcp_result = await session.call_tool(func_name, func_args)
                    result_text = ""
                    for block in mcp_result.content:
                        if hasattr(block, "text"): result_text += block.text
                    print(f"  📥 Result received ({len(result_text)} chars)")
                    preview = result_text[:200].replace('\n', ' ')
                    print(f"  📄 Preview: {preview}...")
                except Exception as e:
                    result_text = json.dumps({"error": str(e)})
                    print(f"  ❌ Error: {e}")
                messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": result_text})
        elif choice.finish_reason == "stop":
            print(f"  ✅ LLM finished!")
            return message.content or "No response", stats
        else:
            print(f"  ⚠️  Unexpected: {choice.finish_reason}")
            if message.content: return message.content, stats
            messages.append(message.model_dump())
    return "Max iterations reached", stats

# ── Create MCP server file (needed for demos to work) ──
os.makedirs('mcp_server', exist_ok=True)

_SERVER_CODE = r'''
import json, sys
from datetime import datetime
try:
    from fastmcp import FastMCP
except ImportError as e:
    print(f"Import error: {e}", file=sys.stderr)
    sys.exit(1)

mcp = FastMCP("Travel Planning Tools")

FLIGHT_DATABASE = {
    ("SFO", "TYO"): [
        {"airline": "ANA", "flight": "NH7", "depart": "11:30 AM", "arrive": "3:30 PM +1", "duration": "11h 00m", "price": 820, "stops": 0, "window_seat": True},
        {"airline": "JAL", "flight": "JL1", "depart": "1:00 PM", "arrive": "5:15 PM +1", "duration": "11h 15m", "price": 910, "stops": 0, "window_seat": True},
        {"airline": "United", "flight": "UA837", "depart": "2:45 PM", "arrive": "7:00 PM +1", "duration": "11h 15m", "price": 750, "stops": 0, "window_seat": False},
        {"airline": "Delta (via Seoul)", "flight": "DL295", "depart": "9:00 AM", "arrive": "8:30 PM +1", "duration": "17h 30m", "price": 620, "stops": 1, "window_seat": True},
    ],
    ("TYO", "SFO"): [
        {"airline": "ANA", "flight": "NH8", "depart": "5:00 PM", "arrive": "10:30 AM same day", "duration": "9h 30m", "price": 840, "stops": 0, "window_seat": True},
        {"airline": "JAL", "flight": "JL2", "depart": "6:30 PM", "arrive": "11:45 AM same day", "duration": "9h 15m", "price": 890, "stops": 0, "window_seat": True},
        {"airline": "United", "flight": "UA838", "depart": "4:00 PM", "arrive": "9:30 AM same day", "duration": "9h 30m", "price": 770, "stops": 0, "window_seat": True},
    ],
}

HOTEL_DATABASE = {
    "Tokyo": [
        {"name": "K5 Tokyo", "area": "Nihonbashi", "type": "boutique", "rating": 4.8, "price_night": 190, "vegetarian": True},
        {"name": "Trunk Hotel", "area": "Shibuya", "type": "boutique", "rating": 4.6, "price_night": 220, "vegetarian": True},
        {"name": "MUJI Hotel Ginza", "area": "Ginza", "type": "boutique", "rating": 4.5, "price_night": 175, "vegetarian": True},
        {"name": "APA Hotel Shinjuku", "area": "Shinjuku", "type": "business", "rating": 3.8, "price_night": 95, "vegetarian": False},
        {"name": "Park Hyatt Tokyo", "area": "Shinjuku", "type": "luxury", "rating": 4.9, "price_night": 550, "vegetarian": True},
    ]
}

CALENDAR = [
    {"date": "2026-10-01", "time": "09:00-10:00", "event": "Team standup", "flexible": True},
    {"date": "2026-10-03", "time": "14:00-15:00", "event": "Dentist appointment", "flexible": False},
    {"date": "2026-10-07", "time": "11:00-12:00", "event": "Project review", "flexible": False},
]

USER_PREFS = {
    "name": "Alex Johnson", "home_airport": "SFO",
    "seat": "window", "meal": "vegetarian", "hotel_type": "boutique",
    "budget_style": "moderate",
    "loyalty": ["ANA Mileage Club", "Marriott Bonvoy"],
    "interests": ["Japanese cuisine", "temples", "street photography", "local markets"],
    "past_trips": ["Kyoto 2023", "Osaka 2024"],
}

TOKYO_TIPS = {
    "best_months": ["March-April (Cherry Blossom)", "October-November (Autumn)"],
    "must_visit": ["Senso-ji Temple", "Meiji Shrine", "Tsukiji Market", "Shibuya Crossing"],
    "vegetarian_spots": ["Ain Soph Ripple (Shinjuku)", "T's TanTan (Tokyo Station)", "Bon (Taito)"],
    "tip": "Get a 72-hour Tokyo Metro Pass for 1,500 yen (~$10)",
}

@mcp.tool()
def search_flights(origin: str, destination: str, date: str) -> str:
    """Search flights between two airports on a specific date. Returns real pricing and availability."""
    origin = origin.upper().strip()
    destination = destination.upper().strip()
    code_map = {"TOKYO": "TYO", "NRT": "TYO", "HND": "TYO", "SAN FRANCISCO": "SFO"}
    origin = code_map.get(origin, origin)
    destination = code_map.get(destination, destination)
    flights = FLIGHT_DATABASE.get((origin, destination), [])
    results = [dict(f, date=date) for f in flights]
    return json.dumps({"route": f"{origin} -> {destination}", "date": date, "count": len(results), "flights": results, "currency": "USD"}, indent=2)

@mcp.tool()
def search_hotels(city: str, checkin: str, checkout: str, hotel_type: str = "any") -> str:
    """Search hotels in a city with pricing. Filter by type: boutique, business, luxury, or any."""
    hotels = HOTEL_DATABASE.get(city.strip().title(), [])
    if hotel_type != "any":
        hotels = [h for h in hotels if h["type"] == hotel_type.lower()]
    try:
        nights = (datetime.strptime(checkout, "%Y-%m-%d") - datetime.strptime(checkin, "%Y-%m-%d")).days
    except ValueError:
        nights = 5
    results = [dict(h, checkin=checkin, checkout=checkout, nights=nights, total_price=h["price_night"] * nights) for h in hotels]
    results.sort(key=lambda x: x["rating"], reverse=True)
    return json.dumps({"city": city.title(), "checkin": checkin, "checkout": checkout, "nights": nights, "count": len(results), "hotels": results, "currency": "USD"}, indent=2)

@mcp.tool()
def check_calendar(start_date: str, end_date: str) -> str:
    """Check user calendar for events and conflicts in a date range."""
    try:
        d1 = datetime.strptime(start_date, "%Y-%m-%d")
        d2 = datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        return json.dumps({"error": "Invalid date format. Use YYYY-MM-DD"})
    events_in_range = []
    conflicts = []
    for ev in CALENDAR:
        ev_date = datetime.strptime(ev["date"], "%Y-%m-%d")
        if d1 <= ev_date <= d2:
            events_in_range.append(ev)
            if not ev["flexible"]:
                conflicts.append(ev)
    return json.dumps({"range": f"{start_date} to {end_date}", "events": events_in_range, "conflicts": conflicts, "conflict_count": len(conflicts)}, indent=2)

@mcp.tool()
def estimate_daily_expenses(city: str, style: str = "moderate") -> str:
    """Estimate daily expenses for a city. Style: budget, moderate, or luxury."""
    costs = {"Tokyo": {"budget": {"food": 30, "transport": 10, "activities": 20, "misc": 10}, "moderate": {"food": 60, "transport": 15, "activities": 40, "misc": 20}, "luxury": {"food": 150, "transport": 50, "activities": 100, "misc": 50}}}
    city_costs = costs.get(city.title(), {}).get(style.lower())
    if not city_costs:
        return json.dumps({"error": f"No data for {city}/{style}"})
    city_costs["total_per_day"] = sum(city_costs.values())
    return json.dumps({"city": city.title(), "style": style, "daily_usd": city_costs}, indent=2)

@mcp.tool()
def calculate_trip_budget(flight_cost: float, hotel_cost: float, daily_expense: float, num_days: int, total_budget: float) -> str:
    """Calculate total trip cost and check if it fits within budget."""
    daily_total = daily_expense * num_days
    total = flight_cost + hotel_cost + daily_total
    remaining = total_budget - total
    return json.dumps({"breakdown": {"flights": flight_cost, "hotel": hotel_cost, "daily_expenses": daily_total, "total": total}, "budget": total_budget, "remaining": remaining, "feasible": remaining >= 0, "verdict": f"Within budget! ${remaining:.0f} left." if remaining >= 0 else f"Over by ${abs(remaining):.0f}."}, indent=2)

@mcp.resource("user://travel-preferences")
def get_user_preferences() -> str:
    """User travel preferences and profile."""
    return json.dumps(USER_PREFS, indent=2)

@mcp.resource("data://tokyo-travel-tips")
def get_tokyo_tips() -> str:
    """Local travel tips for Tokyo."""
    return json.dumps(TOKYO_TIPS, indent=2)

@mcp.prompt()
def trip_planner_prompt(destination: str, days: int, budget: float) -> str:
    """System prompt for comprehensive trip planning."""
    return f"""You are an expert travel planner. Plan a {days}-day trip to {destination} within ${budget}.
Steps:
1. Read user://travel-preferences for personalization
2. Use check_calendar to find scheduling conflicts
3. Use search_flights for real flight options
4. Use search_hotels matching user hotel type preference
5. Use estimate_daily_expenses for cost estimation
6. Use calculate_trip_budget to verify feasibility
7. Compile a comprehensive, personalized plan
IMPORTANT: Use REAL data from tools. Never make up prices."""

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''.strip()

with open('mcp_server/travel_server.py', 'w') as f:
    f.write(_SERVER_CODE)

KIWI_MCP_URL = 'https://mcp.kiwi.com/mcp'

SERVER_PARAMS = StdioServerParameters(
    command=sys.executable,
    args=['mcp_server/travel_server.py'],
    env=None,
)

USER_QUERY = '''
Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.
I prefer window seats, vegetarian food, and boutique hotels.
Search real flights and hotels with actual prices.
Book everything if it looks good.
'''

print(f'✅ All setup complete | Model: {MODEL}')
print(f'✅ MCP server file created at mcp_server/travel_server.py')


✅ All setup complete | Model: gpt-5.2
✅ MCP server file created at mcp_server/travel_server.py


---
# 🎯 SECTION A: See the Magic
Just run each cell and watch the results. No code reading needed!

## Step 1: Our travel request

In [ ]:
print('💬 Our request to the AI:\n')
print(USER_QUERY)

💬 Our request to the AI:


Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.
I prefer window seats, vegetarian food, and boutique hotels.
Search real flights and hotels with actual prices.
Book everything if it looks good.



## Step 2: Ask AI *without* any tools

In [ ]:
print_header('AI Response WITHOUT Tools')
print('\n🧠 Asking the AI with NO tools...\n')

start_time = time.time()
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {'role': 'system', 'content': 'You are a travel planning assistant. Answer the user\'s travel planning request.'},
        {'role': 'user', 'content': USER_QUERY},
    ],
)
raw_answer = response.choices[0].message.content
print(f'⏱️  Response in {time.time() - start_time:.1f}s\n')
print(raw_answer)


════════════════════════════════════════════════════════════
  AI Response WITHOUT Tools
════════════════════════════════════════════════════════════

🧠 Asking the AI with NO tools...

⏱️  Response in 26.9s

I can absolutely plan the 5-day Tokyo trip (October 2026) within $3,000 from San Francisco with vegetarian-friendly options and boutique hotels—but I can’t **search live inventory/prices** or **book** flights/hotels from here because I don’t have access to real-time airline/hotel systems or your payment details. What I *can* do is:

- Build a complete itinerary and a short list of **high-probability** flight/hotel options that usually fit your preferences/budget  
- Tell you **exactly what to click/filter** to find the real prices quickly (Google Flights/airline sites + Booking/Agoda)  
- If you paste links/screenshots of flight/hotel results, I’ll compare them and tell you which to book (and what seat/room rate to choose)

To lock this down, I need a few specifics:

### 1) Quick 

☝️ Notice the problem? The AI made up everything. No real data was used.

In [ ]:
print_header('🔍 What went wrong?')
problems = [
    ('Prices are MADE UP',       'No API was called'),
    ('No real availability',      'It doesn\'t know if those flights/hotels exist'),
    ('No calendar awareness',     'It doesn\'t know your schedule'),
    ('Cannot actually book',      'It\'s just text'),
    ('No personalization',        'Doesn\'t know your preferences'),
    ('No budget validation',      'Never checked if the plan fits $3000'),
]
for i, (problem, detail) in enumerate(problems, 1):
    print(f'\n  ❌ Problem {i}: {problem}')
    print(f'     → {detail}')
print(f'\n{"═" * 55}')
print('  💡 SOLUTION: Give the AI real tools via MCP!')
print(f'{"═" * 55}')


════════════════════════════════════════════════════════════
  🔍 What went wrong?
════════════════════════════════════════════════════════════

  ❌ Problem 1: Prices are MADE UP
     → No API was called

  ❌ Problem 2: No real availability
     → It doesn't know if those flights/hotels exist

  ❌ Problem 3: No calendar awareness
     → It doesn't know your schedule

  ❌ Problem 4: Cannot actually book
     → It's just text

  ❌ Problem 5: No personalization
     → Doesn't know your preferences

  ❌ Problem 6: No budget validation
     → Never checked if the plan fits $3000

═══════════════════════════════════════════════════════
  💡 SOLUTION: Give the AI real tools via MCP!
═══════════════════════════════════════════════════════


## Step 3: Give the AI a real flight search tool (Kiwi.com)

In [ ]:
print_header('AI + Real Flight Search (Kiwi.com MCP)')

KIWI_QUERY = '''
Search for real flights from San Francisco (SFO) to Tokyo
for October 1st 2026 to October 6th 2026.
Show me the cheapest options with actual prices.
'''
print(f'💬 Query: {KIWI_QUERY.strip()}\n')

async def run_kiwi_demo():
    async with streamablehttp_client(url=KIWI_MCP_URL) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools_response = await session.list_tools()
            openai_tools = mcp_tools_to_openai_functions(tools_response.tools)
            print(f'🔧 {len(openai_tools)} tools available to AI\n')
            messages = [
                {'role': 'system', 'content': 'You are a travel assistant with access to real flight search tools. Use the available tools to search for REAL flights. Never make up prices.'},
                {'role': 'user', 'content': KIWI_QUERY},
            ]
            start = time.time()
            answer, stats = await run_agentic_loop(session, messages, openai_tools)
            return answer, stats, time.time() - start

kiwi_answer, kiwi_stats, kiwi_time = await run_kiwi_demo()


════════════════════════════════════════════════════════════
  AI + Real Flight Search (Kiwi.com MCP)
════════════════════════════════════════════════════════════
💬 Query: Search for real flights from San Francisco (SFO) to Tokyo
for October 1st 2026 to October 6th 2026.
Show me the cheapest options with actual prices.

🔧 2 tools available to AI


🔄 Iteration 1
─────────────────────────────────────────────
  ⏱️  LLM responded in 1.8s | Finish: tool_calls
  🔧 LLM wants 1 tool call(s)

  📤 TOOL CALL: search-flight
     flyFrom: SFO
     flyTo: Tokyo
     departureDate: 01/10/2026
     returnDate: 06/10/2026
     passengers: {'adults': 1, 'children': 0, 'infants': 0}
     cabinClass: M
     sort: price
     curr: USD
     locale: en
  📥 Result received (19369 chars)
  📄 Preview: [   {     "flyFrom": "SFO",     "flyTo": "NRT",     "cityFrom": "San Francisco",     "cityTo": "Tokyo",     "departure": {       "utc": "2026-10-01T23:45:00.000Z",       "local": "2026-10-01T16:45:00....

🔄 Itera

### 👇 Real prices from a real API!

In [ ]:
print_header('✈️ REAL Flight Results (via Kiwi.com)')
print(f'\n  Tool calls: {kiwi_stats["tool_calls"]}  |  Time: {kiwi_time:.1f}s\n')
print(kiwi_answer)


════════════════════════════════════════════════════════════
  ✈️ REAL Flight Results (via Kiwi.com)
════════════════════════════════════════════════════════════

  Tool calls: 1  |  Time: 12.8s

Here are **real round‑trip economy flights** from **San Francisco (SFO) → Tokyo (NRT/HND)** for **Oct 1, 2026 – Oct 6, 2026**, sorted by **cheapest** (prices in **USD**).

## Cheapest options (actual prices)

| Outbound (airports / layovers) | Outbound (local times) | Cabin | Return (airports / layovers) | Return (local times) | Cabin | Price (USD) | Booking link |
|---|---:|---|---|---:|---|---:|---|
| **SFO → NRT** | 10/01 16:45 → 10/02 20:00 (**11h 15m**) | Economy | **NRT → SFO** | 10/06 21:30 → 10/06 14:45 (**9h 15m**) | Economy | **$792** | https://on.kiwi.com/UyODSP |
| **SFO → HKG → NRT** | 10/01 13:00 → 10/03 15:55 (**34h 55m**) | Economy | **NRT → HKG → SFO** | 10/06 17:15 → 10/06 22:40 (**21h 25m**) | Economy | **$808** | https://on.kiwi.com/ix295R |
| **SFO → HKG → NRT** | 10/01 1

## Step 4: Give the AI a full toolkit — flights, hotels, calendar, budget
Watch the AI automatically figure out which tools to call and in what order.

In [ ]:
print_header('AI + Full Travel Toolkit (5 tools + user preferences + travel tips)')
print(f'💬 Query: {USER_QUERY.strip()}\n')

async def run_full_demo():
    devnull = open(os.devnull, 'w') if os.name == 'posix' else None
    try:
        async with stdio_client(SERVER_PARAMS, errlog=devnull) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                tools_result = await session.list_tools()
                openai_tools = mcp_tools_to_openai_functions(tools_result.tools)
                print(f'🔧 {len(openai_tools)} tools loaded')
                prefs = await session.read_resource('user://travel-preferences')
                prefs_text = prefs.contents[0].text
                tips = await session.read_resource('data://tokyo-travel-tips')
                tips_text = tips.contents[0].text
                print(f'📄 User preferences & travel tips loaded')
                prompt_result = await session.get_prompt('trip_planner_prompt', arguments={'destination': 'Tokyo', 'days': '5', 'budget': '3000'})
                system_prompt = prompt_result.messages[0].content.text
                full_system = f'{system_prompt}\n\n## User Preferences:\n{prefs_text}\n\n## Tokyo Travel Tips:\n{tips_text}'
                messages = [
                    {'role': 'system', 'content': full_system},
                    {'role': 'user', 'content': USER_QUERY},
                ]
                print(f'\n{"═" * 55}')
                print(f'  Starting Agentic Loop...')
                print(f'{"═" * 55}')
                start = time.time()
                answer, stats = await run_agentic_loop(session, messages, openai_tools)
                return answer, stats, time.time() - start
    finally:
        if devnull: devnull.close()

final_answer, final_stats, final_time = await run_full_demo()


════════════════════════════════════════════════════════════
  AI + Full Travel Toolkit (5 tools + user preferences + travel tips)
════════════════════════════════════════════════════════════
💬 Query: Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.
I prefer window seats, vegetarian food, and boutique hotels.
Search real flights and hotels with actual prices.
Book everything if it looks good.

🔧 5 tools loaded
📄 User preferences & travel tips loaded

═══════════════════════════════════════════════════════
  Starting Agentic Loop...
═══════════════════════════════════════════════════════

🔄 Iteration 1
─────────────────────────────────────────────
  ⏱️  LLM responded in 1.1s | Finish: tool_calls
  🔧 LLM wants 1 tool call(s)

  📤 TOOL CALL: check_calendar
     start_date: 2026-10-01
     end_date: 2026-10-31
  📥 Result received (745 chars)
  📄 Preview: {   "range": "2026-10-01 to 2026-10-31",   "events": [     {       "date": "2026-10-01"

### 👇 The complete, personalized travel plan — built from real data

In [ ]:
print_header('📋 COMPLETE TRAVEL PLAN')
print(f'\n  Iterations: {final_stats["iterations"]}  |  Tool calls: {final_stats["tool_calls"]}  |  Time: {final_time:.1f}s')
print(f'  Tools used: {", ".join(final_stats["tools_used"])}\n')
print(final_answer)


════════════════════════════════════════════════════════════
  📋 COMPLETE TRAVEL PLAN
════════════════════════════════════════════════════════════

  Iterations: 5  |  Tool calls: 9  |  Time: 30.8s
  Tools used: search_flights, search_hotels, estimate_daily_expenses, calculate_trip_budget, check_calendar

## Best October 2026 window-seat + vegetarian + boutique plan (5 days / 4 hotel nights) — **within $3000**

### 1) Calendar check (October 2026)
Conflicts found:
- **Oct 3** — Dentist appointment (not flexible)
- **Oct 7** — Project review (not flexible)

To avoid these, I’m proposing travel **Oct 4–9, 2026** (Tokyo time will be Oct 5–9).

---

## 2) Real flight options (SFO ⇄ Tokyo) with prices
I searched both HND/NRT; results are listed as **TYO** options (Tokyo airports).

### Recommended (meets loyalty + window preference): **ANA (nonstop)**
- **Outbound:** **ANA NH7** — **Oct 4, 2026** — Dep **11:30 AM** / Arr **3:30 PM +1** — **Nonstop** — **$820** — *window seat available*
- *

## Step 5: The big picture

In [ ]:
print_header('🏆 COMPARISON: Without Tools vs With Tools')
comparison = [
    ('Real prices',       '❌ Made up',          '✅ Kiwi.com API',  '✅ Our tools'),
    ('Tools used',        '0',                   f'{kiwi_stats["tool_calls"]}',  f'{final_stats["tool_calls"]}'),
    ('User preferences',  '❌ None',             '❌ None',          '✅ Yes'),
    ('Calendar check',    '❌ None',             '❌ None',          '✅ Yes'),
    ('Budget validation', '❌ None',             '❌ None',          '✅ Yes'),
    ('Hotel search',      '❌ Generic',          '❌ N/A',           '✅ Yes'),
    ('Personalized',      '❌ No',               '❌ No',            '✅ Yes'),
]
print(f'\n  {"Feature":<20} {"No Tools":<18} {"Kiwi MCP":<18} {"Full Toolkit"}')
print(f'  {"─"*20} {"─"*18} {"─"*18} {"─"*18}')
for row in comparison:
    print(f'  {row[0]:<20} {row[1]:<18} {row[2]:<18} {row[3]}')
print(f'\n💡 The AI DISCOVERED the tools, DECIDED which to call,')
print(f'   and produced a GROUNDED response with real data.')
print(f'   We didn\'t write any orchestration logic!')
print(f'\n{"═" * 55}')
print(f'  🔮 Want to know HOW this works? Continue to Section B!')
print(f'{"═" * 55}')


════════════════════════════════════════════════════════════
  🏆 COMPARISON: Without Tools vs With Tools
════════════════════════════════════════════════════════════

  Feature              No Tools           Kiwi MCP           Full Toolkit
  ──────────────────── ────────────────── ────────────────── ──────────────────
  Real prices          ❌ Made up          ✅ Kiwi.com API     ✅ Our tools
  Tools used           0                  1                  9
  User preferences     ❌ None             ❌ None             ✅ Yes
  Calendar check       ❌ None             ❌ None             ✅ Yes
  Budget validation    ❌ None             ❌ None             ✅ Yes
  Hotel search         ❌ Generic          ❌ N/A              ✅ Yes
  Personalized         ❌ No               ❌ No               ✅ Yes

💡 The AI DISCOVERED the tools, DECIDED which to call,
   and produced a GROUNDED response with real data.
   We didn't write any orchestration logic!

═══════════════════════════════════════════════════════

---
# 🔬 SECTION B: Under the Hood
Now let's see how each piece works in detail.

## B1: Discovering tools on an external MCP server (Kiwi.com)
When we connect to an MCP server, we first *discover* what tools it offers.

In [ ]:
print_header('Discovering Kiwi.com MCP Tools')

async def discover_kiwi_tools():
    async with streamablehttp_client(url=KIWI_MCP_URL) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            print('✅ Connected to Kiwi.com MCP server!\n')
            tools_response = await session.list_tools()
            print(f'📋 Found {len(tools_response.tools)} tools:\n')
            for i, tool in enumerate(tools_response.tools, 1):
                desc = (tool.description or 'No description')[:80]
                print(f'  {i}. 🔧 {tool.name}')
                print(f'     {desc}\n')
            return tools_response.tools

kiwi_tools = await discover_kiwi_tools()

## B2: Our custom MCP server — the code
This server file was auto-created during setup. It defines 5 tools, 2 resources, and 1 prompt template.

In [ ]:
print_header('Our MCP Server Code')
server_path = os.path.abspath('mcp_server/travel_server.py')
print(f'📄 File: {server_path}\n')
with open(server_path) as f:
    content = f.read()
tools_count = content.count('@mcp.tool()')
resources_count = content.count('@mcp.resource(')
prompts_count = content.count('@mcp.prompt()')
print(f'📦 Contents:  🔧 {tools_count} Tools  |  📄 {resources_count} Resources  |  📝 {prompts_count} Prompt\n')
print(content)

## B3: Verify the server starts without errors

In [ ]:
result = subprocess.run([sys.executable, 'mcp_server/travel_server.py'], capture_output=True, text=True, timeout=5, input='')
if result.returncode == 0 or 'Traceback' not in result.stderr:
    print('✅ Server starts cleanly!')
else:
    print(f'❌ Server has errors:\n{result.stderr}')

## B4: Connect to our server and discover everything it exposes

In [ ]:
print_header('Connecting to Our Custom MCP Server')

async def discover_custom_tools():
    devnull = open(os.devnull, 'w') if os.name == 'posix' else None
    try:
        async with stdio_client(SERVER_PARAMS, errlog=devnull) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                print('✅ Server spawned and connected!\n')
                tools_result = await session.list_tools()
                print(f'🔧 TOOLS ({len(tools_result.tools)}):')
                for t in tools_result.tools:
                    print(f'   • {t.name}: {(t.description or "")[:60]}')
                resources_result = await session.list_resources()
                print(f'\n📄 RESOURCES ({len(resources_result.resources)}):')
                for r in resources_result.resources:
                    print(f'   • {r.uri}')
                prompts_result = await session.list_prompts()
                print(f'\n📝 PROMPTS ({len(prompts_result.prompts)}):')
                for p in prompts_result.prompts:
                    print(f'   • {p.name}: {(p.description or "")[:60]}')
                return tools_result.tools
    finally:
        if devnull: devnull.close()

custom_tools = await discover_custom_tools()

## B5: Reading MCP Resources — user preferences & travel tips

In [ ]:
print_header('Reading MCP Resources')

async def read_resources():
    devnull = open(os.devnull, 'w') if os.name == 'posix' else None
    try:
        async with stdio_client(SERVER_PARAMS, errlog=devnull) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                print('📄 Resource: user://travel-preferences\n')
                prefs = await session.read_resource('user://travel-preferences')
                prefs_text = prefs.contents[0].text if prefs.contents else '{}'
                prefs_data = json.loads(prefs_text)
                print(f'   👤 Name:     {prefs_data.get("name")}')
                print(f'   🏠 Airport:  {prefs_data.get("home_airport")}')
                print(f'   💺 Seat:     {prefs_data.get("seat")}')
                print(f'   🍱 Meal:     {prefs_data.get("meal")}')
                print(f'   🏨 Hotel:    {prefs_data.get("hotel_type")}')
                print(f'   🎯 Interests: {", ".join(prefs_data.get("interests", []))}')
                print(f'\n📄 Resource: data://tokyo-travel-tips\n')
                tips = await session.read_resource('data://tokyo-travel-tips')
                tips_text = tips.contents[0].text if tips.contents else '{}'
                tips_data = json.loads(tips_text)
                print(f'   🗓️  Best months: {", ".join(tips_data.get("best_months", []))}')
                print(f'   📍 Must visit:  {", ".join(tips_data.get("must_visit", []))}')
                print(f'   🍱 Veg spots:   {", ".join(tips_data.get("vegetarian_spots", []))}')
                print(f'   💡 Tip:         {tips_data.get("tip")}')
                return prefs_text, tips_text
    finally:
        if devnull: devnull.close()

user_prefs_text, tokyo_tips_text = await read_resources()

## B6: Getting the MCP Prompt Template

In [ ]:
print_header('Getting MCP Prompt Template')

async def get_prompt_template():
    devnull = open(os.devnull, 'w') if os.name == 'posix' else None
    try:
        async with stdio_client(SERVER_PARAMS, errlog=devnull) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                result = await session.get_prompt('trip_planner_prompt', arguments={'destination': 'Tokyo', 'days': '5', 'budget': '3000'})
                prompt_text = ''
                for msg in result.messages:
                    if hasattr(msg.content, 'text'):
                        prompt_text = msg.content.text
                print('📝 Generated System Prompt:\n')
                print(f'   {prompt_text}')
                return prompt_text
    finally:
        if devnull: devnull.close()

system_prompt = await get_prompt_template()

## B7: Testing each tool individually

In [ ]:
print_header('Testing Individual MCP Tool Calls')

async def test_tools():
    devnull = open(os.devnull, 'w') if os.name == 'posix' else None
    try:
        async with stdio_client(SERVER_PARAMS, errlog=devnull) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                print('🔧 Test: search_flights')
                result = await session.call_tool('search_flights', {'origin': 'SFO', 'destination': 'TYO', 'date': '2026-10-01'})
                flights = json.loads(result.content[0].text)
                print(f'   ✅ Found {flights["count"]} flights')
                for f in flights['flights'][:2]:
                    print(f'      ✈️  {f["airline"]} {f["flight"]} — ${f["price"]}')
                print(f'\n🔧 Test: check_calendar')
                result = await session.call_tool('check_calendar', {'start_date': '2026-10-01', 'end_date': '2026-10-06'})
                cal = json.loads(result.content[0].text)
                print(f'   ✅ Found {len(cal["events"])} events, {cal["conflict_count"]} conflicts')
                for c in cal['conflicts']:
                    print(f'      ⚠️  {c["date"]} {c["time"]}: {c["event"]}')
                print(f'\n🔧 Test: search_hotels')
                result = await session.call_tool('search_hotels', {'city': 'Tokyo', 'checkin': '2026-10-01', 'checkout': '2026-10-06', 'hotel_type': 'boutique'})
                hotels = json.loads(result.content[0].text)
                print(f'   ✅ Found {hotels["count"]} boutique hotels ({hotels["nights"]} nights)')
                for h in hotels['hotels'][:2]:
                    print(f'      🏨 {h["name"]} — ${h["price_night"]}/night (${h["total_price"]} total)')
                print(f'\n🔧 Test: estimate_daily_expenses')
                result = await session.call_tool('estimate_daily_expenses', {'city': 'Tokyo', 'style': 'moderate'})
                expenses = json.loads(result.content[0].text)
                daily = expenses['daily_usd']
                print(f'   ✅ Daily estimate: ${daily["total_per_day"]}/day')
                print(f'\n🔧 Test: calculate_trip_budget')
                result = await session.call_tool('calculate_trip_budget', {'flight_cost': 820, 'hotel_cost': 950, 'daily_expense': 135, 'num_days': 5, 'total_budget': 3000})
                budget = json.loads(result.content[0].text)
                print(f'   ✅ {budget["verdict"]}')
    finally:
        if devnull: devnull.close()

await test_tools()
print('\n✅ All 5 tools working!')

## B8: Architecture — what we built

In [ ]:
print_header('🏗️ Architecture')
print('''
    +------------------+
    |      USER        |
    |  "Plan my trip"  |
    +--------+---------+
             |
             v
    +---------------------------------------+
    |           LLM (GPT)                   |
    |                                       |
    |   Reads:   Resources (prefs/tips)     |
    |   Uses:    Prompt template            |
    |   Calls:   Tools via MCP              |
    +-------+----------+-----------+--------+
            |          |           |
         MCP|       MCP|        MCP|
            v          v           v
    +-----------+ +---------+ +----------+
    | search_   | | check_  | |calculate_|
    | flights   | |calendar | |trip_     |
    | search_   | |         | |budget    |
    | hotels    | |         | |estimate_ |
    +-----------+ +---------+ |expenses  |
                               +----------+
''')
print('═' * 55)
print('  KEY TAKEAWAYS')
print('═' * 55)
takeaways = [
    ('MCP = USB-C for AI', 'Standard protocol to plug tools/data into any LLM'),
    ('3 MCP primitives', 'Tools (actions), Resources (data), Prompts (templates)'),
    ('LLM decides the workflow', 'We wrote ZERO orchestration logic'),
    ('Server is independent', 'Runs as separate process, any language'),
    ('Composable', 'Mix our server + Kiwi.com + any other MCP server'),
]
for title, desc in takeaways:
    print(f'\n  💡 {title}')
    print(f'     {desc}')
print(f'\n\n{"═" * 55}')
print('  WHAT NEXT?')
print('═' * 55)
print('  One agent is powerful. But what about MULTIPLE agents')
print('  working together?')
print('  MCP = Agent <-> Tools  (vertical integration)')
print('  A2A = Agent <-> Agent  (horizontal collaboration)')
print('═' * 55)

# 🎉 Demo Complete!

| Part | What | Result |
|------|------|--------|
| **No Tools** | Raw LLM call | ❌ Hallucinated |
| **Kiwi MCP** | LLM + external MCP server | ✅ Real flight data |
| **Full Toolkit** | LLM + custom MCP (5 tools, 2 resources, 1 prompt) | ✅ Complete grounded plan |

The LLM discovered tools, decided which to call, combined results — we wrote zero orchestration logic!